In [1]:
# Import dependencies.
from citipy import citipy
import pandas as pd
import requests
import numpy as np
import time


# Import API key.
from config import weather_api_key

In [2]:
# Create a set of 2000 random latitude and longitude combinations. 
lats = np.random.uniform(low=-90, high=90, size=2000)
lngs = np.random.uniform(low=-180, high=180, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Assign the latitudes and longitudes to a list. 
coordinates = list(lat_lngs)

# Display coordinates for review.
for coordinate in coordinates:
    print(coordinate[0], coordinate[1])

48.02400484168223 -11.01803828953166
-23.554903260979586 162.06392480624692
63.31570038073335 -82.37557326144724
33.1556819599829 -152.44621512571706
76.94787017908561 -145.45090722098166
89.43262863783269 12.9581512782905
22.28493546383902 146.83329653932975
-57.56601837407379 -88.8619040271316
63.04922788981986 39.41057439226057
12.151749248650049 -108.99518136916282
-70.54482443907962 -147.6551465574102
80.31319135839513 44.999071919983436
38.012014955093235 149.4296923870391
-82.38673610394956 123.10539517098266
-36.8049179419496 -12.238998402319936
1.4884670163747131 -50.71354526510041
-59.110091801447275 -7.312634768250035
82.92282371068691 -108.19826447047072
-15.961188621065318 -28.19596905533416
-25.992116035380675 -153.62282266418663
4.587666777954823 -96.05145335818787
-55.06919336249927 109.49441143153979
-89.07157393544296 -42.35534576599764
45.71433979874601 11.326167387065595
-86.72176153059523 3.251735998092869
-14.733360468779921 28.40631832581812
-37.44603127321239 98

52.554443032847274 -55.2874777574998
63.95598111397578 36.4434717423193
-49.43068515268541 54.174855972771525
-57.00438131434582 148.2234604261689
0.7596874134793978 -31.310009875542676
66.57965886531059 136.05425111172025
0.5500364160922686 102.89076149954838
-68.81336036071289 144.38606221177633
-28.99479324229928 29.76273160277853
36.84121771535587 150.25033094300352
37.314218548066506 -58.74505853878374
-8.03572144037986 129.7367501329872
-21.239003821899047 91.73025505911477
15.830732164783313 162.8075950784164
-38.62035175678891 18.51256976022421
1.4351280182458197 149.009034296096
-77.88548343088827 -132.7741044238888
-39.01916805638022 131.065218565573
39.91420024863669 16.294917319785668
10.32362991546141 -15.833770643866046
34.06257824644227 -20.16763601385597
-20.703785589042596 80.03565899532407
-76.18305619680243 -151.5260062632142
-67.8581042577618 63.67582887487919
79.89543291059368 87.85646476184104
82.29893318277297 -21.597090972943192
-31.012431324858895 46.9380189877

-63.842540568208854 132.3699805654598
-9.78562809775238 66.81846742626479
78.92184360732045 -107.1244815256324
-80.05004569063506 -48.29556471193041
-61.53400688110841 165.52832771195511
78.36804678410414 -29.214772806259134
13.44587866737939 84.79521501106808
81.97725666499551 -142.12075490294427
-51.56211606598854 -61.13633106810029
-62.80422835373497 -120.21239857282538
88.42151964129107 140.07223354369364
53.80394343380584 172.49631770162893
-85.18347663743116 75.7678335381326
-3.667910388136022 168.49979887527059
-54.835613556504285 -48.60114848090873
14.773952876975102 177.5773126315122
-38.949249739283744 62.71283163615891
17.53022077664785 126.50771767782868
12.996783064675427 -170.28851886972228
-53.84944928610118 7.274443575571155
75.3249298124476 -54.143346961128856
-64.12179230936708 -12.903675588697581
8.343939390702786 40.965243141269866
88.87472273831003 -95.47657886419196
-63.82361012458021 104.13016792720146
49.929602234843486 1.6827764958164835
85.39836418758006 107.7

In [4]:
# Create a list for the cities.
cities = []

# Find the nearest city for the latitude and longitude combinations.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    # If city is not a duplicate, add to the cities list.
    if city not in cities:
        cities.append(city)

#Print the city count.
len(cities)

737

In [5]:
# URL for OpenWeatherMap.
url = "https://api.openweathermap.org/data/2.5/weather?units=Imperial&appid="+ weather_api_key

In [6]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Data retrieval initiated     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities.
for i, city in enumerate(cities):

    # Group cities in sets of 50 to avoid errors/timeouts.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} -> {city}")
    # Add 1 to the record count.
    record_count += 1
# Run an API request for each city.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Data to collect.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_desc = city_weather["weather"]
        # Add the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                         "Lat": city_lat,
                         "Lng": city_lng,
                         "Max Temp": city_max_temp,
                         "Humidity": city_humidity,
                         "Cloudiness": city_clouds,
                         "Wind Speed": city_wind,
                         "Current Description": city_desc[0]["description"]})

# If error occurs, skip the city.
    except:
        print("City not found. Skipped...")
        pass

# Indicate that list is complete.
print("-----------------------------")
print("Data retrieval complete      ")
print("-----------------------------")

Data retrieval initiated     
-----------------------------
Processing Record 1 of Set 1 -> skibbereen
Processing Record 2 of Set 1 -> voh
Processing Record 3 of Set 1 -> attawapiskat
City not found. Skipped...
Processing Record 4 of Set 1 -> kailua
Processing Record 5 of Set 1 -> college
Processing Record 6 of Set 1 -> barentsburg
City not found. Skipped...
Processing Record 7 of Set 1 -> katsuura
Processing Record 8 of Set 1 -> punta arenas
Processing Record 9 of Set 1 -> oksovskiy
Processing Record 10 of Set 1 -> san patricio
Processing Record 11 of Set 1 -> mataura
Processing Record 12 of Set 1 -> belushya guba
City not found. Skipped...
Processing Record 13 of Set 1 -> nemuro
Processing Record 14 of Set 1 -> new norfolk
Processing Record 15 of Set 1 -> jamestown
Processing Record 16 of Set 1 -> amapa
Processing Record 17 of Set 1 -> cape town
Processing Record 18 of Set 1 -> yellowknife
Processing Record 19 of Set 1 -> maceio
Processing Record 20 of Set 1 -> avera
Processing Recor

Processing Record 34 of Set 4 -> awbari
Processing Record 35 of Set 4 -> rio grande
Processing Record 36 of Set 4 -> nizhneyansk
City not found. Skipped...
Processing Record 37 of Set 4 -> guerrero negro
Processing Record 38 of Set 4 -> wanaka
Processing Record 39 of Set 4 -> lompoc
Processing Record 40 of Set 4 -> oussouye
Processing Record 41 of Set 4 -> tirat karmel
Processing Record 42 of Set 4 -> waddan
Processing Record 43 of Set 4 -> cherskiy
Processing Record 44 of Set 4 -> svetlyy
Processing Record 45 of Set 4 -> harnosand
Processing Record 46 of Set 4 -> bethel
Processing Record 47 of Set 4 -> saleaula
City not found. Skipped...
Processing Record 48 of Set 4 -> leningradskiy
Processing Record 49 of Set 4 -> wakkanai
Processing Record 50 of Set 4 -> port elizabeth
Processing Record 1 of Set 5 -> den helder
Processing Record 2 of Set 5 -> kyzyl-suu
Processing Record 3 of Set 5 -> moussoro
Processing Record 4 of Set 5 -> el arenal
Processing Record 5 of Set 5 -> eureka
Processin

Processing Record 20 of Set 8 -> gat
Processing Record 21 of Set 8 -> voyvozh
Processing Record 22 of Set 8 -> qarqin
Processing Record 23 of Set 8 -> terrell
Processing Record 24 of Set 8 -> dongsheng
Processing Record 25 of Set 8 -> jiddah
City not found. Skipped...
Processing Record 26 of Set 8 -> bossangoa
Processing Record 27 of Set 8 -> huarmey
Processing Record 28 of Set 8 -> morris
Processing Record 29 of Set 8 -> novoalekseyevskaya
Processing Record 30 of Set 8 -> conakry
Processing Record 31 of Set 8 -> hauterive
Processing Record 32 of Set 8 -> octeville
Processing Record 33 of Set 8 -> opuwo
Processing Record 34 of Set 8 -> altay
Processing Record 35 of Set 8 -> husavik
Processing Record 36 of Set 8 -> san luis
Processing Record 37 of Set 8 -> caslav
Processing Record 38 of Set 8 -> lunenburg
Processing Record 39 of Set 8 -> yaounde
Processing Record 40 of Set 8 -> ayan
Processing Record 41 of Set 8 -> saint anthony
Processing Record 42 of Set 8 -> seguela
Processing Record

Processing Record 6 of Set 12 -> iki-burul
Processing Record 7 of Set 12 -> muros
Processing Record 8 of Set 12 -> najran
Processing Record 9 of Set 12 -> bridlington
Processing Record 10 of Set 12 -> cap-chat
Processing Record 11 of Set 12 -> bowen
Processing Record 12 of Set 12 -> alihe
Processing Record 13 of Set 12 -> staryy nadym
Processing Record 14 of Set 12 -> tombouctou
Processing Record 15 of Set 12 -> ust-bolsheretsk
City not found. Skipped...
Processing Record 16 of Set 12 -> thinadhoo
Processing Record 17 of Set 12 -> batemans bay
Processing Record 18 of Set 12 -> zhanaozen
Processing Record 19 of Set 12 -> te anau
Processing Record 20 of Set 12 -> manasa
Processing Record 21 of Set 12 -> esso
Processing Record 22 of Set 12 -> deputatskiy
Processing Record 23 of Set 12 -> taltal
Processing Record 24 of Set 12 -> kamaishi
Processing Record 25 of Set 12 -> saryshagan
City not found. Skipped...
Processing Record 26 of Set 12 -> geraldton
Processing Record 27 of Set 12 -> ingh

In [7]:
# Confirm city count.
len(city_data)

682

In [8]:
# Convert the data to DataFrame.
city_weather_df = pd.DataFrame(city_data)
city_weather_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Skibbereen,IE,51.5500,-9.2667,63.10,94,100,15.50,overcast clouds
1,Voh,NC,-20.9667,164.7000,71.35,89,63,1.45,broken clouds
2,Kailua,US,21.4022,-157.7394,76.98,80,100,11.50,overcast clouds
3,College,US,64.8569,-147.8028,59.58,93,100,3.44,overcast clouds
4,Katsuura,JP,35.1333,140.3000,80.20,93,100,8.32,overcast clouds


In [12]:
# Create the CSV file.
output_data_file = "../Weather_Database/WeatherPy_Database.csv"

# Export the data into CSV.
city_weather_df.to_csv(output_data_file)